In [133]:
import pyaudio
import numpy as np
import whisper
import openai
import psutil
import platform
import time
import requests
import cv2
import sounddevice as sd
from collections import deque
import openai
from openai import OpenAI

In [134]:
# ================= CONFIGURATION =================
class Config:
    # 1. Use RAW token from @BotFather (no extra characters)
    TELEGRAM_TOKEN = "xxxxxxxxxxxxx"  # ← Replace with NEW token
    CHAT_ID = "xxxxxxxxxxxx"
    # 2. Add strict validation
    @staticmethod
    def validate_token(token):
        parts = token.split(':')
        return len(parts) == 2 and parts[0].isdigit() and len(token) == 46
   
    
    # OpenAI Settings
    DEEPSEEK_KEY = "xxxxxxxxxxxx" # replace with your api key
    MODEL = "deepseek"
    
    # Audio Settings
    SAMPLE_RATE = 16000
    CHUNK_SIZE = 1024
    BUFFER_SECONDS = 3
    
    # Device Check Interval (seconds)
    DEVICE_CHECK_INTERVAL = 5
    
    # Question Triggers
    QUESTION_WORDS = ["what", "how", "why", "tell", "explain", "describe", 
                     "walk me through", "give an example", "difference between"]
print(f"Token type: {type(Config.TELEGRAM_TOKEN)}")
print(f"Token length: {len(Config.TELEGRAM_TOKEN)}")  # Should be 46 characters
print(f"First 10 chars: {Config.TELEGRAM_TOKEN[:10]}")

Token type: <class 'str'>
Token length: 46
First 10 chars: 8267824796


In [135]:
def test_token():
    token = "8403447023:AAEI3VJR_mhERJruSN-B6ElBb6FsZjF0YCI"  # Manually typed
    print(requests.get(f"https://api.telegram.org/bot{token}/getMe").json())

In [136]:
# ================= QUESTION DATABASE =================
QUESTION_DB = {
    # Personal Questions
    "tell me about yourself": 
        "I'm a results-driven professional with expertise in...",
    "walk me through your resume": 
        "My career began at [Company] where I developed...",
    
    # Technical Questions
    "explain oop principles": 
        "Object-Oriented Programming has four pillars: Encapsulation...",
    "what is polymorphism": 
        "Polymorphism allows objects to take many forms through...",
    
    # Behavioral Questions
    "describe a challenge": 
        "In my last project, we faced [Challenge]. I solved it by...",
    "example of leadership": 
        "When leading [Project], I implemented [Strategy] resulting in...",
    
    # Corporate Questions
    "why should we hire you": 
        "I bring [Skills] that directly address your need for...",
    "where do you see yourself": 
        "In 5 years, I aim to grow into [Position] leveraging..."
}

In [137]:
# ================= DEVICE MONITOR =================
class DeviceMonitor:
    def __init__(self):
        self.camera_active = False
        self.mic_active = False
        
    def check_camera(self):
        """Check if camera is being accessed by any application"""
        cap = cv2.VideoCapture(0)
        if cap.isOpened():
            self.camera_active = True
            cap.release()
        else:
            self.camera_active = False
            
    def check_microphone(self):
        """Check if microphone is in use"""
        try:
            # Test recording for 1 second
            fs = 44100
            recording = sd.rec(int(1 * fs), samplerate=fs, channels=1)
            sd.wait()
            self.mic_active = True
        except:
            self.mic_active = False

In [138]:
# ================= AUDIO PROCESSOR =================
class AudioEngine:
    def __init__(self):
        self.model = whisper.load_model("tiny.en")
        self.audio_interface = pyaudio.PyAudio()
        self.stream = None
        self.buffer = np.zeros(0, dtype=np.float32)
        
    def start(self):
        self.stream = self.audio_interface.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=Config.SAMPLE_RATE,
            input=True,
            frames_per_buffer=Config.CHUNK_SIZE,
            stream_callback=self._callback
        )
    
    def _callback(self, in_data, frame_count, time_info, status):
        audio_data = np.frombuffer(in_data, dtype=np.int16)
        self.buffer = np.append(self.buffer, audio_data.astype(np.float32) / 32768.0)
        return (in_data, pyaudio.paContinue)
    
    def process_audio(self):
        if len(self.buffer) >= Config.SAMPLE_RATE * Config.BUFFER_SECONDS:
            result = self.model.transcribe(
                self.buffer,
                fp16=False,
                language='en'
            )
            self.buffer = self.buffer[int(Config.SAMPLE_RATE*1.5):]  # Keep 1.5s overlap
            return result["text"].strip()
        return ""

# ================= SYSTEM MONITOR =================
class SystemMonitor:
    @staticmethod
    def is_interview_active():
        """Check if target apps are running"""
        for proc in psutil.process_iter(['name']):
            if proc.info['name'] and any(
                app in proc.info['name'].lower() 
                for app in ["zoom", "teams", "webex", "hirevue"]
            ):
                return True
        return False

In [139]:
# ================= AI PROCESSOR =================
class InterviewAI:
    def __init__(self):
        self.api_key = Config.DEEPSEEK_KEY  # Change to your DeepSeek key
        self.base_url = "https://api.deepseek.com/v1"  # DeepSeek's API endpoint
        self.conversation_history = deque(maxlen=3)

    def generate_answer(self, question):
        # Check predefined answers first (unchanged)
        question_lower = question.lower()
        for q, a in QUESTION_DB.items():
            if q in question_lower:
                return a

        # Fallback to DeepSeek API
        try:
            headers = {
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            }
            
            # Format messages with history
            messages = [{"role": "system", "content": "You are an interview coach. Answer concisely in 2 sentences max."}]
            for q, a in self.conversation_history:
                messages.extend([
                    {"role": "user", "content": q},
                    {"role": "assistant", "content": a}
                ])
            messages.append({"role": "user", "content": question})

            data = {
                "model": "deepseek-chat",  # or "deepseek-coder" for technical questions
                "messages": messages,
                "temperature": 0.3,
                "max_tokens": 150
            }

            response = requests.post(
                f"{self.base_url}/chat/completions",
                headers=headers,
                json=data,
                timeout=10
            )
            
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                print(f"⚠️ DeepSeek API Error: {response.text}")
                return "[Answer generation failed]"

        except Exception as e:
            print(f"⚠️ DeepSeek connection error: {str(e)}")
            return "[Service unavailable]"

In [140]:
# ================= TELEGRAM INTEGRATION =================
class TelegramBot:
    @staticmethod
    def send(question, answer):
        try:
            # 1. Get and validate token
            token = Config.TELEGRAM_TOKEN.strip()
            if not Config.validate_token(token):
                raise ValueError(f"Invalid token format: {token[:10]}... (Expected '1234567890:ABC...')")

            # 2. Test connection
            test_url = f"https://api.telegram.org/bot{token}/getMe"
            test_res = requests.get(test_url, timeout=5)
            
            if not test_res.json().get('ok'):
                raise ConnectionError(f"Token rejected. Full response: {test_res.text}")

            # 3. Send message
            payload = {
                "chat_id": Config.CHAT_ID,
                "text": f"🎤 Q: {question}\n\n🤖 A: {answer}",
                "parse_mode": "Markdown"
            }
            response = requests.post(
                f"https://api.telegram.org/bot{token}/sendMessage",
                json=payload,
                timeout=10
            )
            
            if response.status_code == 200:
                print(f"✅ Telegram confirmed message delivery!")
                return True
                
            raise ConnectionError(f"Telegram error {response.status_code}: {response.text}")
            
        except Exception as e:
            print(f"🚨 CRITICAL FAILURE: {str(e)}")
            return False

In [141]:
# ================= MAIN CONTROLLER =================
class InterviewNinja:
    def __init__(self):
        self.audio = AudioEngine()
        self.monitor = SystemMonitor()
        self.device_monitor = DeviceMonitor()
        self.ai = InterviewAI()
        self.active = False
        self.last_device_check = 0
        
    def run(self):
        print("🕵️‍♂️ AI Interview Ninja started in stealth mode")
        print("🔍 Monitoring for interview activity...")
        self.audio.start()
        
        try:
            while True:
                # Check devices periodically
                if time.time() - self.last_device_check > Config.DEVICE_CHECK_INTERVAL:
                    self.device_monitor.check_camera()
                    self.device_monitor.check_microphone()
                    self.last_device_check = time.time()
                
                # Activation logic
                if not self.active:
                    if (self.monitor.is_interview_active() or 
                        self.device_monitor.camera_active or 
                        self.device_monitor.mic_active):
                        self.active = True
                        print("🚀 Interview detected! Activating ninja mode...")
                        TelegramBot.send("System", "Ninja mode activated 👁️")
                else:
                    if not (self.monitor.is_interview_active() or 
                          self.device_monitor.camera_active or 
                          self.device_monitor.mic_active):
                        self.active = False
                        print("🌑 Interview ended. Going dark...")
                
                # Process audio when active
                if self.active:
                    text = self.audio.process_audio()
                    if text and self._is_question(text):
                        print(f"❓ Detected question: {text}")
                        answer = self.ai.generate_answer(text)
                        TelegramBot.send(text, answer)
                        self.ai.conversation_history.append((text, answer))
                
                time.sleep(0.1)  # Reduce CPU usage
                
        except KeyboardInterrupt:
            self.audio.stream.stop_stream()
            self.audio.stream.close()
            self.audio.audio_interface.terminate()
            print("🛑 System shutdown complete")

    def _is_question(self, text):
        """Enhanced question detection"""
        text_lower = text.lower()
        return (
            "?" in text or
            any(text_lower.startswith(word) for word in Config.QUESTION_WORDS) or
            any(word in text_lower for word in ["interview", "resume", "experience"]) or
            len(text.split()) > 4
        )

In [142]:
if __name__ == "__main__":
    # First test Telegram directly
    print("Testing Telegram connection...")
    TelegramBot.send("Test", "If you see this, Telegram works!")
    
    # Then test OpenAI
    print("Testing OpenAI...")
    ai = InterviewAI()
    test_answer = ai.generate_answer("Tell me about yourself")
    print(f"OpenAI test: {test_answer}")
    
    # Finally run system
    print("Starting main system...")
    ninja = InterviewNinja()
    ninja.run()

Testing Telegram connection...
🚨 CRITICAL FAILURE: Token rejected. Full response: {"ok":false,"error_code":401,"description":"Unauthorized"}
Testing OpenAI...
OpenAI test: I'm a results-driven professional with expertise in...
Starting main system...
🕵️‍♂️ AI Interview Ninja started in stealth mode
🔍 Monitoring for interview activity...
🚀 Interview detected! Activating ninja mode...
🚨 CRITICAL FAILURE: Token rejected. Full response: {"ok":false,"error_code":401,"description":"Unauthorized"}
❓ Detected question: . . . . . . . . . . . . . . . . . . . .
⚠️ DeepSeek API Error: {"error":{"message":"Insufficient Balance","type":"unknown_error","param":null,"code":"invalid_request_error"}}
🚨 CRITICAL FAILURE: Token rejected. Full response: {"ok":false,"error_code":401,"description":"Unauthorized"}
❓ Detected question: Is thank you. a one passed
⚠️ DeepSeek API Error: {"error":{"message":"Insufficient Balance","type":"unknown_error","param":null,"code":"invalid_request_error"}}
🚨 CRITICAL FAIL